In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs
import json
from time import sleep

In [4]:
election_id = 478

In [8]:
all_constituencies = pd.read_csv('data/constituencies.csv')
all_constituencies.head()

,zillaID,constituencyID,constituency_name
0,4,125,ঝালোকাঠি-১
1,4,126,ঝালোকাঠি-২
2,5,111,পটুয়াখালী-১
3,5,112,পটুয়াখালী-২
4,5,113,পটুয়াখালী-৩


In [9]:
dfcand = pd.read_csv('data/candidates.csv')
dfcand.head()

,zillaID,constituencyID,name,img,party,affidavit,election_fund,tax_return
0,4,125,মোঃ সাব্বির আহমেদ,https://onss.ecs.gov.bd/document-service/api/v...,স্বতন্ত্র,মোঃ সাব্বির আহমেদ_হলফনামা | https://onss.ecs.g...,মোঃ সাব্বির আহমেদ_নির্বাচনী ব্যয় ও ব্যক্তিগত স...,মোঃ সাব্বির আহমেদ_আয়কর রিটার্ন | https://onss....
1,4,125,মুহাম্মদ মুস্তাফিজুর রহমান,https://onss.ecs.gov.bd/document-service/api/v...,বাংলাদেশ লেবার পার্টি,মুহাম্মদ মুস্তাফিজুর রহমান_হলফনামা | https://o...,মুহাম্মদ মুস্তাফিজুর রহমান_নির্বাচনী ব্যয় ও ব্...,মুহাম্মদ মুস্তাফিজুর রহমান_আয়কর রিটার্ন | http...
2,4,125,মাহমুদা আলম মিতু,https://onss.ecs.gov.bd/document-service/api/v...,জাতীয় নাগরিক পার্টি-এনসিপি,মাহমুদা আলম মিতু_হলফনামা | https://onss.ecs.go...,মাহমুদা আলম মিতু_নির্বাচনী ব্যয় ও ব্যক্তিগত সম...,মাহমুদা আলম মিতু_আয়কর রিটার্ন | https://onss.e...
3,4,125,"এম, মঈন আলম ফিরোজী",https://onss.ecs.gov.bd/document-service/api/v...,স্বতন্ত্র,"এম, মঈন আলম ফিরোজী_হলফনামা | https://onss.ecs....","এম, মঈন আলম ফিরোজী_নির্বাচনী ব্যয় ও ব্যক্তিগত ...","এম, মঈন আলম ফিরোজী_আয়কর রিটার্ন | https://onss..."
4,4,125,রফিকুল ইসলাম জামাল,https://onss.ecs.gov.bd/document-service/api/v...,বাংলাদেশ জাতীয়তাবাদী দল - বি.এন.পি,রফিকুল ইসলাম জামাল_হলফনামা | https://onss.ecs....,রফিকুল ইসলাম জামাল_নির্বাচনী ব্যয় ও ব্যক্তিগত ...,রফিকুল ইসলাম জামাল_আয়কর রিটার্ন | https://onss...


In [10]:
# get finalized candidate info links
candidates = []
for _, cons in all_constituencies.iterrows():
    zillaID = cons['zillaID']
    constituencyID = cons['constituencyID']
    url = f'http://103.183.38.66/get/candidate/data?zilla_id={zillaID}&candidate_type=1&election_id={election_id}&constituency_id={constituencyID}&status_id=11'
    print(f'fetching constituency =', cons['constituency_name'])
    res = requests.get(url)
    soup = bs(res.text, 'html.parser')
    # print(soup.prettify())
    rows = soup.select('tr')
    print(f'candidates =', len(rows))
    for row in rows:
        cols = row.select('td')
        candidate_data = [zillaID, constituencyID]
        for col in cols:
            img = col.select_one('img')
            if img:
                continue
            a = col.select_one('a')
            if a:
                continue
            text = col.text.strip()
            if text != '':
                candidate_data.append(text)
                continue
        candidates.append(candidate_data)
    sleep(2)
# print(candidates)

fetching constituency = ঝালোকাঠি-১
candidates = 10
fetching constituency = ঝালোকাঠি-২
candidates = 8
fetching constituency = পটুয়াখালী-১
candidates = 6
fetching constituency = পটুয়াখালী-২
candidates = 5
fetching constituency = পটুয়াখালী-৩
candidates = 5
fetching constituency = পটুয়াখালী-৪
candidates = 4
fetching constituency = পিরোজপুর-১
candidates = 2
fetching constituency = পিরোজপুর-২
candidates = 7
fetching constituency = পিরোজপুর-৩
candidates = 6
fetching constituency = বরগুনা-১
candidates = 4
fetching constituency = বরগুনা-২
candidates = 9
fetching constituency = বরিশাল-১
candidates = 5
fetching constituency = বরিশাল-২
candidates = 8
fetching constituency = বরিশাল-৩
candidates = 6
fetching constituency = বরিশাল-৪
candidates = 5
fetching constituency = বরিশাল-৫
candidates = 6
fetching constituency = বরিশাল-৬
candidates = 6
fetching constituency = ভোলা-১
candidates = 8
fetching constituency = ভোলা-২
candidates = 8
fetching constituency = ভোলা-৩
candidates = 5
fetching constituency =

In [11]:
len(candidates)

2009

In [31]:
df = pd.DataFrame(candidates)
df.head()

,0,1,2,3,4,5
0,4,125,মোঃ সাব্বির আহমেদ,স্বতন্ত্র,মোরগ,None
1,4,125,এম. মঈন আলম ফিরোজী,স্বতন্ত্র,হাঁস,None
2,4,125,রফিকুল ইসলাম জামাল,বাংলাদেশ জাতীয়তাবাদী দল - বি.এন.পি,ধানের শীষ,None
3,4,125,মোঃ জসীম উদ্দিন তালুকদার,জনতার দল,কলম,None
4,4,125,মোঃ কামরুজ্জামান খান,জাতীয় পার্টি,লাঙ্গল,None


In [32]:
df.columns = ['zillaID', 'constituencyID', 'name', 'party', 'symbol', 'what?']
df.head()

,zillaID,constituencyID,name,party,symbol,what?
0,4,125,মোঃ সাব্বির আহমেদ,স্বতন্ত্র,মোরগ,None
1,4,125,এম. মঈন আলম ফিরোজী,স্বতন্ত্র,হাঁস,None
2,4,125,রফিকুল ইসলাম জামাল,বাংলাদেশ জাতীয়তাবাদী দল - বি.এন.পি,ধানের শীষ,None
3,4,125,মোঃ জসীম উদ্দিন তালুকদার,জনতার দল,কলম,None
4,4,125,মোঃ কামরুজ্জামান খান,জাতীয় পার্টি,লাঙ্গল,None


In [33]:
df['what?'].value_counts()

what?
ফাইল পাওয়া যায় নি    1
Name: count, dtype: int64

In [34]:
df.symbol.value_counts()

symbol
ধানের শীষ      288
হাতপাখা        256
দাঁড়িপাল্লা    226
লাঙ্গল         197
ট্রাক           92
              ... 
সোফা             1
টেলিফোন          1
রেল ইঞ্জিন       1
মাইক             1
মোবাইল ফোন       1
Name: count, Length: 77, dtype: int64

In [35]:
dfcand['final'] = False
dfcand['symbol'] = None
for i, row in df.iterrows():
    symbol = row['symbol']
    dfcand.loc[(dfcand['constituencyID']==row['constituencyID']) & (dfcand['name']==row['name']), ['symbol', 'final']] = [symbol, True]

dfcand.head()

,zillaID,constituencyID,name,img,party,affidavit,election_fund,tax_return,final,symbol
0,4,125,মোঃ সাব্বির আহমেদ,https://onss.ecs.gov.bd/document-service/api/v...,স্বতন্ত্র,মোঃ সাব্বির আহমেদ_হলফনামা | https://onss.ecs.g...,মোঃ সাব্বির আহমেদ_নির্বাচনী ব্যয় ও ব্যক্তিগত স...,মোঃ সাব্বির আহমেদ_আয়কর রিটার্ন | https://onss....,True,মোরগ
1,4,125,মুহাম্মদ মুস্তাফিজুর রহমান,https://onss.ecs.gov.bd/document-service/api/v...,বাংলাদেশ লেবার পার্টি,মুহাম্মদ মুস্তাফিজুর রহমান_হলফনামা | https://o...,মুহাম্মদ মুস্তাফিজুর রহমান_নির্বাচনী ব্যয় ও ব্...,মুহাম্মদ মুস্তাফিজুর রহমান_আয়কর রিটার্ন | http...,False,None
2,4,125,মাহমুদা আলম মিতু,https://onss.ecs.gov.bd/document-service/api/v...,জাতীয় নাগরিক পার্টি-এনসিপি,মাহমুদা আলম মিতু_হলফনামা | https://onss.ecs.go...,মাহমুদা আলম মিতু_নির্বাচনী ব্যয় ও ব্যক্তিগত সম...,মাহমুদা আলম মিতু_আয়কর রিটার্ন | https://onss.e...,False,None
3,4,125,"এম, মঈন আলম ফিরোজী",https://onss.ecs.gov.bd/document-service/api/v...,স্বতন্ত্র,"এম, মঈন আলম ফিরোজী_হলফনামা | https://onss.ecs....","এম, মঈন আলম ফিরোজী_নির্বাচনী ব্যয় ও ব্যক্তিগত ...","এম, মঈন আলম ফিরোজী_আয়কর রিটার্ন | https://onss...",False,None
4,4,125,রফিকুল ইসলাম জামাল,https://onss.ecs.gov.bd/document-service/api/v...,বাংলাদেশ জাতীয়তাবাদী দল - বি.এন.পি,রফিকুল ইসলাম জামাল_হলফনামা | https://onss.ecs....,রফিকুল ইসলাম জামাল_নির্বাচনী ব্যয় ও ব্যক্তিগত ...,রফিকুল ইসলাম জামাল_আয়কর রিটার্ন | https://onss...,True,ধানের শীষ


In [36]:
dfcand.final.value_counts()

final
True     1890
False     557
Name: count, dtype: int64

In [24]:
dfcand.tail()

,zillaID,constituencyID,name,img,party,affidavit,election_fund,tax_return,final,symbol
2442,32,67,মোঃ আসাদুল হক,https://onss.ecs.gov.bd/document-service/api/v...,আমজনতার দল,মোঃ আসাদুল হক_হলফনামা | https://onss.ecs.gov....,মোঃ আসাদুল হক_নির্বাচনী ব্যয় ও ব্যক্তিগত সম্প...,মোঃ আসাদুল হক_আয়কর রিটার্ন | https://onss.ecs...,True,
2443,32,67,"এস, এম, সাইফ মোস্তাফিজ",https://onss.ecs.gov.bd/document-service/api/v...,জাতীয় নাগরিক পার্টি-এনসিপি,"এস, এম, সাইফ মোস্তাফিজ_হলফনামা | https://onss....","এস, এম, সাইফ মোস্তাফিজ_নির্বাচনী ব্যয় ও ব্যক্ত...","এস, এম, সাইফ মোস্তাফিজ_আয়কর রিটার্ন | https://...",False,None
2444,32,67,ইলোরা খাতুন,https://onss.ecs.gov.bd/document-service/api/v...,জাতীয় সমাজতান্ত্রিক দল-জেএসডি,ইলোরা খাতুন_হলফনামা | https://onss.ecs.gov.bd/...,ইলোরা খাতুন_নির্বাচনী ব্যয় ও ব্যক্তিগত সম্পদের...,ইলোরা খাতুন_আয়কর রিটার্ন | https://onss.ecs.go...,True,
2445,32,67,মোঃ আনোয়ার হোসেন,https://onss.ecs.gov.bd/document-service/api/v...,বাংলাদেশের সমাজতান্ত্রিক দল-বাসদ,মোঃ আনোয়ার হোসেন_হলফনামা | https://onss.ecs...,মোঃ আনোয়ার হোসেন_নির্বাচনী ব্যয় ও ব্যক্তিগত...,মোঃ আনোয়ার হোসেন_আয়কর রিটার্ন | https://ons...,True,
2446,32,67,মোঃ হুমায়ন কবির,https://onss.ecs.gov.bd/document-service/api/v...,স্বতন্ত্র,মোঃ হুমায়ন কবির_হলফনামা | https://onss.ecs.go...,মোঃ হুমায়ন কবির_নির্বাচনী ব্যয় ও ব্যক্তিগত সম...,মোঃ হুমায়ন কবির_আয়কর রিটার্ন | https://onss.e...,True,


In [37]:
dfcand.to_csv('data/candidates.csv', index=False)